In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import statsmodels.api as sm
import warnings
from sklearn.model_selection import train_test_split
from itertools import combinations
from scipy import stats
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_absolute_error
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

file = 'rainfalldata.csv'
rd = pd.read_csv(file)
file2 = 'ncrainfalldata.csv'
ncrd = pd.read_csv(file2)
rd.Date = pd.to_datetime(rd.Date)
rd = rd.set_index('Date')

In [4]:
#from ipynb.fs.full.Data_Wrangling_CAP1 import exofind
#latlong = pd.read_csv('latlong.csv')
#latlongsplit = latlong.iloc[0].apply(str.split, sep=',')
#latlongdf = pd.DataFrame(latlongsplit)
#latlongdf = latlongdf.drop(['Unnamed: 0','Raleigh AP, NC', 'Greensboro, NC', ' WILMINGTON 7 N, NC','LUMBERTON, NC','MYRTLE BEACH, SC','CHARLOTTE DOUGLAS AIRPORT, NC','GRNVL SPART INTL AP, SC','PICKENS, SC',' MT. MITCHELL, NC',' Caesars Head Area, SC'])


SyntaxError: invalid syntax (Data_Wrangling_CAP1.ipynb, line 294)

In [ ]:
# if this doesn't work you can comment out this cell and then run the next cell.
# locations = rd.columns
# ncloc = locations[locations.str.endswith('NC')]
# valoc = locations[locations.str.endswith('VA')]
# scloc = locations[locations.str.endswith('SC')]
# galoc = locations[locations.str.endswith('GA')]
# tnloc = locations[locations.str.endswith('TN')]
# exoloc = valoc.append(galoc)
# exoloc = exoloc.append(scloc)
# exoloc = exoloc.append(tnloc)
# exogen = exofind(latlongdf, ncloc, exoloc)

In [5]:
# this cell takes the stored exogen dictionary that is stored in the Data_Wrangling_CAP1 jupyter notebook
# that was imported above.

%store -r exogen
exogen

{'Albemarle, NC': ['CHESTERFIELD 3 E, SC',
  ' CHERAW, SC',
  'PAGELAND 9.0 WNW, SC',
  'FORT MILL 4 NW, SC'],
 'Arcola, NC': ['John Kerr Dam, VA', 'Emporia, VA'],
 'Burlington, NC': ['MARTINSVILLE FILTER PLANT, VA'],
 'Enfield, NC': ['John Kerr Dam, VA', 'Emporia, VA'],
 'Greensboro AP, NC': ['MARTINSVILLE FILTER PLANT, VA', 'STUART, VA'],
 'Henderson 2 NNW, NC': ['SOUTH BOSTON, VA', 'John Kerr Dam, VA'],
 'JACKSON SPRINGS 5 WNW, NC': ['CHESTERFIELD 3 E, SC', ' CHERAW, SC'],
 'Laurinburg, NC': [' DARLINGTON, SC',
  ' DILLON, SC',
  ' FLORENCE REGIONAL AIRPORT, SC',
  'FLORENCE 8 NE, SC',
  'CHESTERFIELD 3 E, SC',
  ' CHERAW, SC'],
 'Louisburg, NC': ['John Kerr Dam, VA'],
 'Roanoke Rapids, NC': ['John Kerr Dam, VA', 'Emporia, VA'],
 'Rougemont, NC': ['CHATHAM, VA', 'SOUTH BOSTON, VA', 'John Kerr Dam, VA'],
 'Murfreesboro, NC': ['Emporia, VA'],
 'Lumberton Area, NC': [' DILLON, SC', ' LORIS 2 S, SC'],
 'ELIZABETHTOWN, NC': [' DILLON, SC', ' LORIS 2 S, SC'],
 'LONGWOOD, NC': ['CONWAY, SC

In [6]:
def sarima_model_creation(data, p, d, q, P, D, Q, m, exog=None):
    my_order = [p,d,q]
    my_sorder = [P,D,Q,m]
    sarimamod = sm.tsa.statespace.SARIMAX(data, exog, order=my_order, seasonal_order=my_sorder, 
                                          enforce_stationarity=False, enforce_invertibility=False,
                                          initialization='approximate_diffuse')
    model_fit = sarimamod.fit()# start_params=[0, 0, 0, 0, 1])
    return(model_fit)

In [7]:
def iteration_hyper(it):
    outlist = []
    for AR in range(it):
        for MA in range(it):
            for SAR in range(it):
                for SMA in range(it):
                    outlist.append([AR,MA,SAR,SMA])
    return(outlist)
        
config = iteration_hyper(5)

In [8]:
def hyperparameter_find(training_data, comb, testing_data, search = False, exogtr = None, exogtest = None):
    leastmae = 1000
    for com in comb:
        li_one_step = []
        for i in range(len(testing_data)):
            if i == 0:
                copytraining = training_data.copy()
                if exogtr is not None:
                    excopy = exogtr.copy()
                    mod_1 = sarima_model_creation(copytraining, com[0], 0, com[1], com[2], 0, 
                                                  com[3], 12, exog=excopy)
                    one_step_pred = mod_1.forecast(exog=excopy.iloc[[-12]]) #uses the data from the year before
                    excopy = pd.concat([excopy, exogtest.iloc[[i]]])
                else:
                    mod_1 = sarima_model_creation(copytraining, com[0], 0, com[1], com[2], 0, com[3], 12)
                    one_step_pred = mod_1.forecast()
                li_one_step.append(one_step_pred[0])
                copytraining = pd.concat([copytraining, testing_data[[i]]])
            else:
                if exogtr is not None:
                    mod_1 = sarima_model_creation(copytraining, com[0], 0, com[1], com[2], 0, 
                                                  com[3], 12, exog=excopy)
                    one_step_pred2 = mod_1.forecast(exog=excopy.iloc[[-12]])
                    excopy = pd.concat([excopy, exogtest.iloc[[i]]])
                else:
                    mod_1 = sarima_model_creation(copytraining, com[0], 0, com[1], com[2], 0, com[3], 12)
                    one_step_pred2 = mod_1.forecast()
                li_one_step.append(one_step_pred2[0])
                copytraining = pd.concat([copytraining, testing_data[[i]]])
        mae = mean_absolute_error(testing_data, li_one_step)
        if search is True:
            if mae < leastmae:
                leastmae = mae
                H_AR = com[0]
                H_MA = com[1]
                H_SAR = com[2]
                H_SMA = com[3]
            print(com,mae)            
    if search is True:
        return('AR: '+ str(H_AR), 'MA: ' +str(H_MA), 'SAR: '+str(H_SAR), 'SMA: '+str(H_SMA))
    else:
        return(mae)

In [9]:
def exog_combinations(df, exoe):
    lo_dfs = []
    if len(exoe) == 1:
        lo_dfs.append(df.loc[:,exoe])
    if len(exoe) > 1:
        lo_dfs.append(df.loc[:,exoe])
        for ex in exoe:
            lo_dfs.append(df.loc[:,[ex]])
        if len(exoe) >2:
            for i in range(2, len(exoe)):
                combolist = list(combinations(exoe,i))
                for c in combolist:
                    lo_dfs.append(df.loc[:,c])
    return(lo_dfs)

In [10]:
def exogenous_var(data, exo_dict, best_comb):
    for key, value in exo_dict.items():
        dat = data[key]
        l_exog = exog_combinations(data, value)
        tr, test = train_test_split(dat, test_size = 0.2, shuffle=False)
        keymae = hyperparameter_find(tr, best_comb, test)
        print('keymae of: '+ key +' = '+str(keymae))
        bettermae = {}
        for exog in l_exog:
            extr, extest = train_test_split(exog, test_size = 0.2, shuffle=False)
            exmae = hyperparameter_find(tr, best_comb, test, exogtr=extr, exogtest = extest)
            co = tuple(exog.columns)
            print('exmae = {}'.format(co) + ' '+ str(exmae))
            if exmae < keymae:
                bettermae[co] = exmae
                bettermae2 = {key: bettermae}
    return(bettermae2)

In [13]:
best_comb = [[4,3,3,4]]
warnings.filterwarnings("ignore")
exogenous_var(rd, exogen, best_comb)

keymae of: Albemarle, NC = 1.5904008088797692
exmae = ('CHESTERFIELD 3 E, SC', ' CHERAW, SC', 'PAGELAND 9.0 WNW, SC', 'FORT MILL 4 NW, SC') 2.1867810706478417
exmae = ('CHESTERFIELD 3 E, SC',) 2.0026856185560185
exmae = (' CHERAW, SC',) 2.03563420794712
exmae = ('PAGELAND 9.0 WNW, SC',) 2.1243296938897926
exmae = ('FORT MILL 4 NW, SC',) 1.8601769476758798
exmae = ('CHESTERFIELD 3 E, SC', ' CHERAW, SC') 2.0831100923047634
exmae = ('CHESTERFIELD 3 E, SC', 'PAGELAND 9.0 WNW, SC') 2.1519029878150464
exmae = ('CHESTERFIELD 3 E, SC', 'FORT MILL 4 NW, SC') 1.97623831054366
exmae = (' CHERAW, SC', 'PAGELAND 9.0 WNW, SC') 2.136970301004525
exmae = (' CHERAW, SC', 'FORT MILL 4 NW, SC') 2.1552244520445862
exmae = ('PAGELAND 9.0 WNW, SC', 'FORT MILL 4 NW, SC') 2.122994270945512
exmae = ('CHESTERFIELD 3 E, SC', ' CHERAW, SC', 'PAGELAND 9.0 WNW, SC') 2.164044103509995
exmae = ('CHESTERFIELD 3 E, SC', ' CHERAW, SC', 'FORT MILL 4 NW, SC') 2.0865050094296236
exmae = ('CHESTERFIELD 3 E, SC', 'PAGELAND 9

keymae of: Louisburg, NC = 1.1135761168461014
exmae = ('John Kerr Dam, VA',) 1.3815404438954075
keymae of: Roanoke Rapids, NC = 1.9781205009704792
exmae = ('John Kerr Dam, VA', 'Emporia, VA') 2.4392555306784085
exmae = ('John Kerr Dam, VA',) 2.295039964919641
exmae = ('Emporia, VA',) 2.40016977921927
keymae of: Rougemont, NC = 0.14928901785940335
exmae = ('CHATHAM, VA', 'SOUTH BOSTON, VA', 'John Kerr Dam, VA') 0.9904694121074239
exmae = ('CHATHAM, VA',) 0.8776618485215745
exmae = ('SOUTH BOSTON, VA',) 0.9667489544654663
exmae = ('John Kerr Dam, VA',) 0.776885406042303
exmae = ('CHATHAM, VA', 'SOUTH BOSTON, VA') 0.9734781531353356
exmae = ('CHATHAM, VA', 'John Kerr Dam, VA') 0.8991684042993114
exmae = ('SOUTH BOSTON, VA', 'John Kerr Dam, VA') 0.9790380191120857
keymae of: Murfreesboro, NC = 1.7268929361598468
exmae = ('Emporia, VA',) 2.1601787524621536
keymae of: Lumberton Area, NC = 1.6959647478482371
exmae = (' DILLON, SC', ' LORIS 2 S, SC') 1.7641758164709496
exmae = (' DILLON, SC',)

KeyboardInterrupt: 

In [ ]:
# if you need to pause the code from running and then start back up from where you left off you can use the
# following code to get a sub_dictionary that will be only a portion of the large dictionary (exogen) that I am
# using

# I had to do this because my computer started running really slow so I had to restart it in order to let the 
# memory reset and so it could start running at normal speed again.

# use exogen.keys() to list all of the keys then find the location where you left off from the printout above
# and then copy and paste the rest of the keys into the "todokeys" tuple below. Then call sub_exogen in place
# of exogen in the "exogenous_var" function above. 


# todokeys = ('Henderson 2 NNW, NC', 'JACKSON SPRINGS 5 WNW, NC', 'Laurinburg, NC', 'Louisburg, NC', 'Roanoke Rapids, NC', 'Rougemont, NC', 'Murfreesboro, NC', 'Lumberton Area, NC', 'ELIZABETHTOWN, NC', 'LONGWOOD, NC', 'WHITEVILLE 7 NW, NC', ' WILLIAM O HUSKE L&D, NC', 'Asheville Area, NC', 'Charlotte Area, NC', 'Mount Mitchell Area, NC', 'ASHEVILLE AIRPORT, NC', 'BANNER ELK, NC', 'BEECH MOUNTAIN, NC', 'BRYSON CITY 4, NC', 'BLACK MOUNTAIN 2 W, NC', 'BREVARD, NC', 'BRIDGEWATER HYDRO, NC', 'CASAR, NC', 'CATAWBA 3 NNW, NC', 'CONCORD, NC', 'COWEETA EXP STATION, NC', 'CULLOWHEE, NC', 'FOREST CITY 8 W, NC', 'FRANKLIN, NC', 'GASTONIA, NC', 'GRANDFATHER MTN, NC', ' HENDERSONVILLE 1 NE, NC', 'Hickory FAA Airport, NC', ' HIGHLANDS, NC', 'HOT SPRINGS, NC', 'LAKE LURE 2, NC', 'LAKE TOXAWAY 2 SW, NC', 'LENOIR, NC', 'LINCOLNTON 4 W, NC', 'MARION, NC', 'MARSHALL, NC', 'MONROE 2 SE, NC', 'MORGANTON, NC')
# sub_exogen = {k: exogen[k] for k in todokeys}